In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
I = np.array([[1,0],[0,1]])
sx = np.array([[0,1],[1,0]])
sy = np.array([[0,-1j],[1j,0]])
sz = np.array([[1,0],[0,-1]])

sx2 = np.kron(np.kron(I,sx),I)
sz2 = np.kron(np.kron(I,sz),I)

sx_ab = np.kron(np.kron(sx,I),sx)
sy_ab = np.kron(np.kron(sy,I),sy)
sz_ab = np.kron(np.kron(sz,I),sz)

sx_ac = np.kron(np.kron(sx,sx),I)
sy_ac = np.kron(np.kron(sy,sy),I)
sz_ac = np.kron(np.kron(sz,sz),I)

sx_bc = np.kron(np.kron(I,sx),sx)
sy_bc = np.kron(np.kron(I,sy),sy)
sz_bc = np.kron(np.kron(I,sz),sz)

In [10]:
#Basis of subsystem A
a0 = np.array([1,0]).reshape(2,1)
a1 = np.array([0,1]).reshape(2,1)
a0_tilde = np.array([1,0]).reshape(2,1)
a1_tilde = np.array([0,1]).reshape(2,1)

#Basis of subsystem B
b0 = np.array([1,1+1j]).reshape(2,1)
b1 = np.array([1-1j,1]).reshape(2,1)
b0_tilde = np.array([-1,1+1j]).reshape(2,1)
b1_tilde = np.array([1-1j,-1]).reshape(2,1)

#Basis of subsystem C
c0 = a0
c1 = a1
c0_tilde = a0_tilde
c1_tilde = a1_tilde

ket_a0b0c0 = np.kron(a0,np.kron(b0,c0))
ket_a0b0c1 = np.kron(a0,np.kron(b0,c1))
ket_a0b1c0 = np.kron(a0,np.kron(b1,c0))
ket_a0b1c1 = np.kron(a0,np.kron(b1,c1))
ket_a1b0c0 = np.kron(a1,np.kron(b0,c0))
ket_a1b0c1 = np.kron(a1,np.kron(b0,c1))
ket_a1b1c0 = np.kron(a1,np.kron(b1,c0))
ket_a1b1c1 = np.kron(a1,np.kron(b1,c1))

bra_a0b0c0 = np.kron(a0_tilde,np.kron(b0_tilde,c0_tilde))
bra_a0b0c1 = np.kron(a0_tilde,np.kron(b0_tilde,c1_tilde))
bra_a0b1c0 = np.kron(a0_tilde,np.kron(b1_tilde,c0_tilde))
bra_a0b1c1 = np.kron(a0_tilde,np.kron(b1_tilde,c1_tilde))
bra_a1b0c0 = np.kron(a1_tilde,np.kron(b0_tilde,c0_tilde))
bra_a1b0c1 = np.kron(a1_tilde,np.kron(b0_tilde,c1_tilde))
bra_a1b1c0 = np.kron(a1_tilde,np.kron(b1_tilde,c0_tilde))
bra_a1b1c1 = np.kron(a1_tilde,np.kron(b1_tilde,c1_tilde))

In [13]:
def sigmaz_ketbasis(size,idx):   #size = number of particles
    basis = np.zeros(2**size).reshape(-1,1)
    basis[idx] = 1
    return basis

In [3]:
def H(hx,hz,d):
    H = []
    for i in range(len(hz)):
        H_c = 1j*hz[i]*sz2 + hx[i]*sx2
        H_ab = sx_ab + sy_ab + d[i]*sz_ab
        H_bc = sx_bc + sy_bc + d[i]*sz_bc
        H_ac = sx_ac + sy_ac + d[i]*sz_ac
        H_t = H_c + H_ab + H_bc + H_ac
        H.append(H_t)
    return H

In [4]:
def rho_c(ket_psi,ket_phi):
    sigma_z = np.array([[1,0],[0,-1]])
    I = np.array([[1,0],[0,1]])
    Sminus = np.array([[0,0],[1,0]])
    sigma_cz = np.kron(np.kron(I,sigma_z),I)
    Sminus_cz = np.kron(np.kron(I,Sminus),I)
    I_cz = np.kron(np.kron(I,I),I)
    a = 0.5*(np.conj(ket_psi).T @ (I_cz+sigma_cz) @ ket_psi)
    b = np.conj(ket_psi).T @ Sminus_cz @ ket_psi
    rho_rc = np.array([[a,b],[np.conj(b),1-a]]).reshape(2,2)
    
    c = 0.5*(np.conj(ket_phi).T @ (I_cz+sigma_cz) @ ket_phi)
    d = np.conj(ket_phi).T @ Sminus_cz @ ket_phi
    rho_lc = np.array([[c,d],[np.conj(d),1-c]]).reshape(2,2)
    return rho_rc, rho_lc

In [11]:
def partial_trace(rho,idx1,idx2): #idx1(idx2) = no of particles to be traced towards left(right)
    particles = int(np.log2(rho.shape[0]))
    rdm = np.zeros((particles-idx1-idx2,particles-idx1-idx2))
    mid = np.eye(2**(particles-idx1-idx2))
    for i in range(2**idx1):
        for j in range(2**idx2):
            basis1 = sigmaz_ketbasis(idx1,i)
            basis2 = sigmaz_ketbasis(idx2,j)
            rdm = rdm + np.kron(basis1.T,np.kron(mid,basis2.T)) @ rho @ np.kron(basis1,np.kron(mid,basis2))
    return rdm

In [5]:
def Entropy(rdm_c):
    lmbda = np.linalg.eigvals(rdm_c)
    S = 0
    for i in range(len(lmbda)):
        k = lmbda[i]
        if np.round(k,4) == 0 or np.round(k,4) == -0:
            S = S - 0
        else:
            S = S - k*np.log2(k)           
    return S

In [6]:
def S_c(H):
    S_rc = []
    S_lc = []
    for i in range(len(H)):    
        ket_psi, ket_phi = eigvectors(H[i])
        rdm_rc, rdm_lc = rho_c(ket_psi,ket_phi)
        S_rc.append(Entropy(rdm_rc))
        S_lc.append(Entropy(rdm_lc))
    return S_rc, S_lc

In [8]:
def gs_eigvectors(H):
    H_dag = np.conj(H).T
    e,psi = np.linalg.eig(H)
    f,phi = np.linalg.eig(H_dag)
    ket_psi = psi[:,np.argmin(e)].reshape(8,1)
    ket_phi = phi[:,np.argmin(f)].reshape(8,1)
    return ket_psi, ket_phi

In [9]:
def bi_rho_c(dm1):
    p = 0.5*((np.conj(bra_a0b0c0).T @ dm1 @ ket_a0b0c0) + (np.conj(bra_a0b0c1).T @ dm1 @ ket_a0b0c1) 
    + (np.conj(bra_a1b0c0).T @ dm1 @ ket_a1b0c0) + (np.conj(bra_a1b0c1).T @ dm1 @ ket_a1b0c1))
    s = 0.5*((np.conj(bra_a0b1c0).T @ dm1 @ ket_a0b1c0) + (np.conj(bra_a0b1c1).T @ dm1 @ ket_a0b1c1) 
    + (np.conj(bra_a1b1c0).T @ dm1 @ ket_a1b1c0) + (np.conj(bra_a1b1c1).T @ dm1 @ ket_a1b1c1))
    q = 0.5*((np.conj(bra_a0b0c0).T @ dm1 @ ket_a0b1c0) + (np.conj(bra_a0b0c1).T @ dm1 @ ket_a0b1c1) 
    + (np.conj(bra_a1b0c0).T @ dm1 @ ket_a1b1c0) + (np.conj(bra_a1b0c1).T @ dm1 @ ket_a1b1c1))
    r = 0.5*((np.conj(bra_a0b1c0).T @ dm1 @ ket_a0b0c0) + (np.conj(bra_a0b1c1).T @ dm1 @ ket_a0b0c1) 
    + (np.conj(bra_a1b1c0).T @ dm1 @ ket_a1b0c0) + (np.conj(bra_a1b1c1).T @ dm1 @ ket_a1b0c1))
    return np.array([[p,q],[r,s]]).reshape(2,2)